# AgenticVision Data Processing Script:

This file contains the inital prototype for Agentic Vision data processing script.
It is no longer in use.

In [7]:
#Import Libraries:
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import scapy
import subprocess
import os

In [ ]:
#Import labels for raw data
df = pd.read_csv("Datastore/Metadata/labels.csv")
df.head() 

In [14]:
df = pd.read_parquet("G:\AV_Datastore\processed\session_features.parquet")
df.head()

,Session_ID,Workload_Type,User_Profile,num_flows,total_packets,total_bytes,total_forward_bytes,total_backward_bytes,session_ul_dl_ratio,avg_flow_bytes,...,avg_flow_throughput,flow_bytes_std,flow_bytes_max,flow_bytes_min,dominant_flow_pct,avg_iat_mean,avg_burstiness,total_direction_changes,tcp_flow_ratio,is_ai
0,20260204_1222_Web_Browsing_PC_Baseline,Web_Browsing,Baseline,36,735,75937,22562,53375,0.422707,2109.361111,...,19140.316835,6247.319417,36238,121,47.721137,0.645787,1.656576,326,0.333333,0
1,20260204_1907_AI_Image_Gen_PC_AI,AI_Image_Gen,AI,111,1024,98019,27591,70428,0.391762,883.054054,...,512354.583867,2846.118747,24141,54,24.628898,1.109100,0.489703,389,0.180180,1
2,20260204_1917_Youtube_Video_PC_Baseline,Youtube_Video,Baseline,88,719,70136,17789,52347,0.339828,797.000000,...,35542.513350,1981.669730,10605,121,15.120623,0.542400,0.364465,261,0.136364,0


In [13]:
#Remove payload information from raw files
raw_file_names = df["Session_ID"]

def strip_payload(files):
    editcap_path = r"C:\Program Files\Wireshark\editcap.exe" #initalise editcap
    
    for raw_file in files:
        input_file = rf"G:\AV_Datastore\raw\{raw_file}.pcapng"
        output_file = rf"G:\AV_Datastore\payload_removed\{raw_file}.pcapng"
        
        # Make sure input file exists
        if not os.path.exists(input_file):
            print(f"File not found: {input_file}")
            continue
        
        # Run editcap to truncate packets to headers only
        subprocess.run([
            editcap_path,
            "-s", "128",  # truncate packets to 128 bytes
            input_file,
            output_file
        ], check=True)
        print(f"Processed: {raw_file}")

strip_payload(raw_file_names)

Processed: 20260204_1222_Web_Browsing_PC_Baseline
Processed: 20260204_1907_AI_Image_Gen_PC_AI
Processed: 20260204_1917_Youtube_Video_PC_Baseline


In [11]:
#Convert stiped file into flow features: # working verson so dont touch #### just for payload removed to flow file
from scapy.all import rdpcap
from collections import defaultdict
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path

class FlowExtractor:
    """This class extracts flow-level statistics from the stripped .PCAPNG files."""
    
    def __init__(self, pcap_path, source_name):
        self.pcap_path = pcap_path
        self.source_name = source_name
        self.flows = defaultdict(lambda: {
            'packets': [],
            'bytes': [],
            'timestamps': [],
            'directions': []  # 0 = forward, 1 = backward
        })
        
    def extract_flows(self):
        """Parse PCAP and group packets into flows."""
        
        print(f"Reading: {self.pcap_path}")
        packets = rdpcap(self.pcap_path)
        
        # First pass: identify local IP (most common source)
        src_ips = [pkt[1].src for pkt in packets if pkt.haslayer('IP')]
        from collections import Counter
        local_ip = Counter(src_ips).most_common(1)[0][0]
        # print(f"Detected local IP: {local_ip}")
        
        # Second pass: group into flows
        for pkt in packets:
            if not pkt.haslayer('IP'):
                continue
                
            # Extract 5-tuple
            src_ip = pkt['IP'].src
            dst_ip = pkt['IP'].dst
            
            # Get transport layer info
            if pkt.haslayer('TCP'):
                proto = 'TCP'
                src_port = pkt['TCP'].sport
                dst_port = pkt['TCP'].dport
            elif pkt.haslayer('UDP'):
                proto = 'UDP'
                src_port = pkt['UDP'].sport
                dst_port = pkt['UDP'].dport
            else:
                continue
            
            # Normalize flow direction (always local -> remote)
            if src_ip == local_ip:
                flow_key = (src_ip, dst_ip, src_port, dst_port, proto)
                direction = 0  # forward (upload)
            else:
                flow_key = (dst_ip, src_ip, dst_port, src_port, proto)
                direction = 1  # backward (download)
            
            # Add packet to flow
            self.flows[flow_key]['packets'].append(pkt)
            self.flows[flow_key]['bytes'].append(len(pkt))
            self.flows[flow_key]['timestamps'].append(float(pkt.time))
            self.flows[flow_key]['directions'].append(direction)
        
        print(f"Identified {len(self.flows)} unique flows")
        return self.flows
    
    def compute_flow_features(self):
        """Computes the statistical features for each flow."""
        
        if not self.flows:
            self.extract_flows()
        
        flow_features = []
        
        for flow_key, flow_data in self.flows.items():
            src_ip, dst_ip, src_port, dst_port, proto = flow_key
            
            packets = flow_data['packets']
            byte_sizes = np.array(flow_data['bytes'])
            timestamps = np.array(flow_data['timestamps'])
            directions = np.array(flow_data['directions'])
            
            # Basic flow info
            features = {
                'src_ip': src_ip,
                'dst_ip': dst_ip,
                'src_port': src_port,
                'dst_port': dst_port,
                'protocol': proto,}
            
            # Flow-level statistics
            features['num_packets'] = len(packets)
            features['total_bytes'] = byte_sizes.sum()
            features['duration'] = timestamps.max() - timestamps.min()
            
            # Avoid division by zero
            if features['duration'] > 0:
                features['throughput_bps'] = (features['total_bytes'] * 8) / features['duration']
            else:
                features['throughput_bps'] = 0
            
            # Packet size statistics
            features['bytes_mean'] = byte_sizes.mean()
            features['bytes_std'] = byte_sizes.std()
            features['bytes_min'] = byte_sizes.min()
            features['bytes_max'] = byte_sizes.max()
            features['bytes_median'] = np.median(byte_sizes)
            
            # Inter-arrival times
            if len(timestamps) > 1:
                iats = np.diff(timestamps)
                features['iat_mean'] = iats.mean()
                features['iat_std'] = iats.std()
                features['iat_min'] = iats.min()
                features['iat_max'] = iats.max()
                features['iat_median'] = np.median(iats)
            else:
                features['iat_mean'] = 0
                features['iat_std'] = 0
                features['iat_min'] = 0
                features['iat_max'] = 0
                features['iat_median'] = 0
            
            # Burstiness (coefficient of variation)
            if features['iat_mean'] > 0:
                features['burstiness'] = features['iat_std'] / features['iat_mean']
            else:
                features['burstiness'] = 0
            
            # Directionality
            forward_packets = (directions == 0).sum()
            backward_packets = (directions == 1).sum()
            
            features['forward_packets'] = forward_packets
            features['backward_packets'] = backward_packets
            
            forward_bytes = byte_sizes[directions == 0].sum()
            backward_bytes = byte_sizes[directions == 1].sum()
            
            features['forward_bytes'] = forward_bytes
            features['backward_bytes'] = backward_bytes
            
            # Upload/Download ratio
            if backward_bytes > 0:
                features['ul_dl_ratio'] = forward_bytes / backward_bytes
            else:
                features['ul_dl_ratio'] = float('inf') if forward_bytes > 0 else 0
            
            # Direction changes (measure of interactivity)
            if len(directions) > 1:
                direction_changes = np.diff(directions) != 0
                features['direction_changes'] = direction_changes.sum()
            else:
                features['direction_changes'] = 0
            
            # Packet size entropy (diversity of packet sizes)
            from scipy.stats import entropy
            if len(byte_sizes) > 1:
                value_counts = np.bincount(byte_sizes)
                probabilities = value_counts / len(byte_sizes)
                features['size_entropy'] = entropy(probabilities)
            else:
                features['size_entropy'] = 0
            
            # Create class labels for AI/Non-AI flows
            AI_tags = {"AI"}
            tag = self.source_name.split("_")[-1]
            features["label"] = 1 if tag in AI_tags else 0
            features["source_file"] = self.source_name # keep source file name for refernce
            
            flow_features.append(features)
        
        return pd.DataFrame(flow_features)

def process_multiple_pcaps_to_single_df(pcap_files, output_path):
    """Process multiple PCAPs, merge all flows into a single dataframe, save as one Parquet."""
    
    all_flows = [] #
    
    for striped_file in pcap_files:
        pcap_file = rf"G:\AV_Datastore\payload_removed\{striped_file}.pcapng"
        
        if not os.path.exists(pcap_file):
            print(f"Missing file: {pcap_file}")
            continue
        
        extractor = FlowExtractor(
            pcap_path=pcap_file,
            source_name=striped_file)
        
        df_flows = extractor.compute_flow_features()
        all_flows.append(df_flows)
    
    if all_flows:
        merged_df = pd.concat(all_flows, ignore_index=True)
        merged_df.to_parquet(output_path, index=False)
        print(f"Saved {len(merged_df)} flows from {len(all_flows)} files to {output_path}")
        return merged_df
    
    print("No flows processed.")
    return pd.DataFrame()

# Usage
if __name__ == "__main__":
    raw_file_names = df["Session_ID"]  # your list of PCAP base names
    output_parquet = r"G:\AV_Datastore\Processed\all_flows.parquet"
    
    merged_df = process_multiple_pcaps_to_single_df(raw_file_names, output_parquet)
    #print(merged_df.head())

Reading: G:\AV_Datastore\payload_removed\20260204_1222_Web_Browsing_PC_Baseline.pcapng


Identified 36 unique flows
Saved 36 flows from 1 files to G:\AV_Datastore\Processed\all_flows.parquet


In [51]:
from scapy.all import rdpcap
from collections import defaultdict
import pandas as pd
import numpy as np
from datetime import datetime

In [4]:

import pandas as pd
from pathlib import Path
from src.features.extract_flows import FlowExtractor
from src.features.aggregate_flows_to_session import aggregate_flows_to_session
from src.pipeline.pcapng_to_features import process_all_pcaps

process_all_pcaps()








FileNotFoundError: [Errno 2] No such file or directory: 'data/metadata/labels.csv'

In [50]:
df1 = pd.read_parquet(r"G:\AV_Datastore\Processed\all_flows.parquet")
df1.head(30)

,src_ip,dst_ip,src_port,dst_port,protocol,num_packets,total_bytes,duration,throughput_bps,bytes_mean,...,burstiness,forward_packets,backward_packets,forward_bytes,backward_bytes,ul_dl_ratio,direction_changes,size_entropy,label,source_file
0,192.168.1.255,192.168.1.162,58866,42100,UDP,1,121,0.000000,0.000000,121.000000,...,0.000000,0,1,0,121,0.000000,0,0.000000,0,20260204_1222_Web_Browsing_PC_Baseline.pcapng
1,192.168.1.255,192.168.1.162,58866,37344,UDP,1,121,0.000000,0.000000,121.000000,...,0.000000,0,1,0,121,0.000000,0,0.000000,0,20260204_1222_Web_Browsing_PC_Baseline.pcapng
2,239.255.255.250,192.168.1.254,1900,50097,UDP,18,2304,0.253413,72735.032405,128.000000,...,3.978817,0,18,0,2304,0.000000,0,0.000000,0,20260204_1222_Web_Browsing_PC_Baseline.pcapng
3,192.168.1.102,3.77.83.229,51657,443,TCP,24,2088,59.123033,282.529484,87.000000,...,4.675127,12,12,1059,1029,1.029155,14,2.023555,0,20260204_1222_Web_Browsing_PC_Baseline.pcapng
4,192.168.1.102,184.25.192.56,51658,443,TCP,81,8560,59.025163,1160.183159,105.679012,...,8.813244,27,54,1980,6580,0.300912,36,1.154075,0,20260204_1222_Web_Browsing_PC_Baseline.pcapng
5,192.168.1.102,18.165.242.12,51660,443,TCP,47,4636,58.948634,629.157922,98.638298,...,6.630526,18,29,1484,3152,0.470812,18,1.615092,0,20260204_1222_Web_Browsing_PC_Baseline.pcapng
6,192.168.1.102,2.22.46.161,51663,443,TCP,31,2959,58.471331,404.847975,95.451613,...,5.370127,15,16,1223,1736,0.704493,14,1.740088,0,20260204_1222_Web_Browsing_PC_Baseline.pcapng
7,192.168.1.102,192.168.1.254,63741,53,UDP,2,208,0.238453,6978.316978,104.000000,...,0.000000,1,1,80,128,0.625000,1,0.693147,0,20260204_1222_Web_Browsing_PC_Baseline.pcapng
8,192.168.1.102,195.181.165.153,64991,443,TCP,19,1144,90.147029,101.523035,60.210526,...,0.999014,10,9,550,594,0.925926,18,0.691761,0,20260204_1222_Web_Browsing_PC_Baseline.pcapng
9,192.168.1.255,192.168.1.162,58866,54115,UDP,1,121,0.000000,0.000000,121.000000,...,0.000000,0,1,0,121,0.000000,0,0.000000,0,20260204_1222_Web_Browsing_PC_Baseline.pcapng
